In [1]:
import os
import sys
sys.path.append(os.path.join(os.getcwd(), '..'))

import numpy as np
from datasets.generation import *
from utils.ply import *
from utils.pc import *

In [2]:
hp = HyperPlane.random_3d()
hp

HyperPlane(normal=array([-0.11983777,  0.08108967, -0.20849885]), bias=0.1835118387427538)

In [3]:
dataset_path = '/home/chukhran/datasets/completion/shapenet/ShapeNetPointCloud'
categories_id = os.listdir(dataset_path)

In [4]:
selected = categories_id[0]
id_to_category.get(selected)

'chair'

In [5]:
random_sample = np.random.choice(os.listdir(os.path.join(dataset_path, selected)))
random_sample

'27a8827b0ba5d743169bfe841050f4a4.ply'

In [6]:
points, _, _ = load_ply(os.path.join(dataset_path, selected, random_sample))
print(len(points))
plot_3d_point_cloud(points)

10000
Switching backend from module://matplotlib_inline.backend_inline to qtagg...


<Figure size 500x500 with 1 Axes>

In [10]:
min_points = 1024
existing, missing = generate_split_sample(points, min_points)
print(f'existing: {existing.shape}, missing: {missing.shape}')

existing: (2112, 3), missing: (7888, 3)


In [11]:
plot_multipart_3d_point_cloud([existing, missing])

<Figure size 1000x1000 with 1 Axes>

In [18]:
import torch
from torch_cluster.radius import radius_graph


In [20]:
plot_3d_point_cloud(points.numpy(), edges=radius_graph_edges.numpy(), show=True, in_u_sphere=True, title='Radius Graph')

Switching backend from QtAgg to qtagg...


<Figure size 944x709 with 1 Axes>

In [24]:
filename = "airplane/1a04e3eab45ca15dd86060f189eb133_1.ply"
existing_path = '/home/chukhran/datasets/completion/shapenet/ShapeNetPointCloud/slices/existing'
missing_path = '/home/chukhran/datasets/completion/shapenet/ShapeNetPointCloud/slices/missing'
existing, _, _ = load_ply(os.path.join(existing_path, filename))
missing, _, _ = load_ply(os.path.join(missing_path, filename))
print(f'existing: {existing.shape}, missing: {missing.shape}')

existing: (4022, 3), missing: (5978, 3)


In [25]:
plot_multipart_3d_point_cloud([existing, missing])

<Figure size 1000x1000 with 1 Axes>

In [26]:
radius = 1e-2

In [30]:
points = torch.from_numpy(np.concatenate([existing, missing], axis=0))

In [31]:
edges = radius_graph(points, r=radius)


In [33]:
edges, edges.shape

(tensor([[2603, 1454, 2908,  ..., 5482, 4165, 8603],
         [   0,    0,    0,  ..., 9999, 9999, 9999]]),
 torch.Size([2, 65408]))

In [34]:
from scipy.spatial import Delaunay

mesh = Delaunay(points.numpy()).simplices

In [36]:
mesh, mesh.shape

(array([[3398, 1162, 3506, 3927],
        [7505, 7499, 3927, 3232],
        [2605, 3398, 2223, 1291],
        ...,
        [6995, 6955, 5462, 9123],
        [6995, 5462, 9496, 9167],
        [6995, 6955, 5462, 9167]], dtype=int32),
 (60653, 4))